In [1]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

import re

import os
import sqlite3
from sqlite3 import Error

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200) 

### 1: Load & Discover Data

In [2]:
with open('Sample-Superstore.csv', 'rb') as file:
    rawdata = file.read(10000)
    result = chardet.detect(rawdata)
    encoding = result['encoding'] 

df = pd.read_csv('Sample-Superstore.csv', encoding=encoding)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [3]:
df.head().T

,0,1,2,3,4
Row ID,1,2,3,4,5
Order ID,CA-2016-152156,CA-2016-152156,CA-2016-138688,US-2015-108966,US-2015-108966
Order Date,11/8/2016,11/8/2016,6/12/2016,10/11/2015,10/11/2015
Ship Date,11/11/2016,11/11/2016,6/16/2016,10/18/2015,10/18/2015
Ship Mode,Second Class,Second Class,Second Class,Standard Class,Standard Class
Customer ID,CG-12520,CG-12520,DV-13045,SO-20335,SO-20335
Customer Name,Claire Gute,Claire Gute,Darrin Van Huff,Sean O'Donnell,Sean O'Donnell
Segment,Consumer,Consumer,Corporate,Consumer,Consumer
Country,United States,United States,United States,United States,United States
City,Henderson,Henderson,Los Angeles,Fort Lauderdale,Fort Lauderdale


In [4]:
df = df.drop(columns=['Row ID'])

In [5]:
# Rename columns 

df.rename(columns={
    'Order ID': 'PurchaseId',
    'Order Date': 'PurchaseDate',
    'Ship Date': 'ShipDate',
    'Ship Mode': 'ShipMode',  
    'Customer ID': 'CustomerId',
    'Customer Name': 'CustomerName',
    'Segment': 'Segment',
    'Country': 'Country',
    'City': 'City',
    'State': 'State',
    'Postal Code': 'PostalCode',
    'Region': 'Region',
    'Product ID': 'ProductId',
    'Category': 'Category',
    'Sub-Category': 'SubCategory',
    'Product Name': 'ProductName',
    'Sales': 'Sales',
    'Quantity': 'Quantity',
    'Discount': 'Discount',
    'Profit': 'Profit'
}, inplace=True)

In [6]:
df.dtypes

PurchaseId       object
PurchaseDate     object
ShipDate         object
ShipMode         object
CustomerId       object
CustomerName     object
Segment          object
Country          object
City             object
State            object
PostalCode        int64
Region           object
ProductId        object
Category         object
SubCategory      object
ProductName      object
Sales           float64
Quantity          int64
Discount        float64
Profit          float64
dtype: object

In [7]:
df.nunique()

PurchaseId      5009
PurchaseDate    1237
ShipDate        1334
ShipMode           4
CustomerId       793
CustomerName     793
Segment            3
Country            1
City             531
State             49
PostalCode       631
Region             4
ProductId       1862
Category           3
SubCategory       17
ProductName     1850
Sales           5825
Quantity          14
Discount          12
Profit          7287
dtype: int64

### 2: Review columns

In [8]:
# check if entries have the same format in PurchaseId column

pattern = r'^[A-Z]{2}-\d{4}-\d{6}$' #  string starting with 2 uppercase letters, followed by a hyphen, 4 digits, another hyphen, and 6 digits

df[~df['PurchaseId'].str.match(pattern)]

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit


In [9]:
# trim whitespaces

df['PurchaseId'] = df['PurchaseId'].str.strip()

In [10]:
# ensure that letters are uppercase  

df['PurchaseId'] = df['PurchaseId'].str.upper()

In [11]:
# check if there is any unexpected characters or symbols 

df[df['PurchaseId'].str.contains(r'[^A-Z0-9-]')]

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit


In [12]:
# check if the year starts with 20 and followed with same pattern 

df[~df['PurchaseId'].str.contains(r'-20\d{2}-')]

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit


In [13]:
# Convert data types for columns 

df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])  

df['ShipDate'] = pd.to_datetime(df['ShipDate'])  

In [14]:
# ensure that the PurchaseDate is always earlier than the ShipDate 

df[df['PurchaseDate'] > df['ShipDate']]

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit


In [15]:
# check entries where OrderDate and ShipDate is same

df_filtered = df[df['PurchaseDate'] == df['ShipDate']]
df_filtered

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
366,CA-2016-155516,2016-10-21,2016-10-21,Same Day,MK-17905,Michael Kennedy,Corporate,United States,Manchester,Connecticut,6040,East,OFF-BI-10002412,Office Supplies,Binders,Wilson Jones Snap Scratch Pad Binder Tool fo...,23.200,4,0.0,10.4400
367,CA-2016-155516,2016-10-21,2016-10-21,Same Day,MK-17905,Michael Kennedy,Corporate,United States,Manchester,Connecticut,6040,East,OFF-SU-10001225,Office Supplies,Supplies,Staple remover,7.360,2,0.0,0.1472
368,CA-2016-155516,2016-10-21,2016-10-21,Same Day,MK-17905,Michael Kennedy,Corporate,United States,Manchester,Connecticut,6040,East,OFF-ST-10002406,Office Supplies,Storage,Pizazz Global Quick File,104.790,7,0.0,29.3412
369,CA-2016-155516,2016-10-21,2016-10-21,Same Day,MK-17905,Michael Kennedy,Corporate,United States,Manchester,Connecticut,6040,East,FUR-BO-10002545,Furniture,Bookcases,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust...",1043.920,4,0.0,271.4192
657,US-2016-156097,2016-09-19,2016-09-19,Same Day,EH-14125,Eugene Hildebrand,Home Office,United States,Aurora,Illinois,60505,Central,FUR-CH-10001215,Furniture,Chairs,Global Troy Executive Leather Low-Back Tilter,701.372,2,0.3,-50.0980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,CA-2017-146269,2017-10-06,2017-10-06,Same Day,MH-17455,Mark Hamilton,Consumer,United States,Chicago,Illinois,60623,Central,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,290.336,2,0.2,32.6628
9872,CA-2017-146269,2017-10-06,2017-10-06,Same Day,MH-17455,Mark Hamilton,Consumer,United States,Chicago,Illinois,60623,Central,OFF-AR-10004790,Office Supplies,Art,Staples in misc. colors,19.152,2,0.2,1.1970
9928,CA-2016-129630,2016-09-04,2016-09-04,Same Day,IM-15055,Ionia McGrath,Consumer,United States,San Francisco,California,94122,West,FUR-FU-10000260,Furniture,Furnishings,"6"" Cubicle Wall Clock, Black",24.270,3,0.0,8.7372
9929,CA-2016-129630,2016-09-04,2016-09-04,Same Day,IM-15055,Ionia McGrath,Consumer,United States,San Francisco,California,94122,West,TEC-CO-10003763,Technology,Copiers,Canon PC1060 Personal Laser Copier,2799.960,5,0.2,944.9865


In [16]:
df_filtered['ShipMode'].unique()

array(['Same Day'], dtype=object)

In [17]:
df['ShipMode'].unique()

array(['Second Class', 'Standard Class', 'First Class', 'Same Day'],
      dtype=object)

In [18]:
# remove whitespaces, capitlize letters for ShipMode column

df['ShipMode'] = df['ShipMode'].str.strip().str.title()

In [19]:
df['ShipMode'].value_counts()

ShipMode
Standard Class    5968
Second Class      1945
First Class       1538
Same Day           543
Name: count, dtype: int64

In [20]:
# make sure all CustomerId values follow a consistent format 

valid_id_pattern = r'^[A-Z]{2}-\d{5}$' # string starting with 2 uppercase, followed by a hyphen and exactly 5 digits

df[~df['CustomerId'].str.match(valid_id_pattern)].T


,261,701,715,1489,1900,2760,2877,2878,2879,2966,2967,3362,3363,3385,3386,3387,3388,3389,4000,4001,4084,4643,4644,4734,5006,5007,5283,5284,5316,5317,5318,5319,5320,5321,5322,5475,5476,5477,6104,7919,7920,8145,8146
PurchaseId,US-2017-155299,CA-2017-114552,CA-2014-153150,CA-2014-136280,CA-2016-140543,CA-2014-129574,CA-2016-152072,CA-2016-152072,CA-2016-152072,CA-2014-162866,CA-2014-162866,CA-2015-139962,CA-2015-139962,CA-2017-148404,CA-2017-148404,CA-2017-148404,CA-2017-148404,CA-2017-148404,CA-2014-116834,CA-2014-116834,CA-2017-163692,CA-2015-147501,CA-2015-147501,CA-2016-120530,CA-2015-169796,CA-2015-169796,CA-2014-133424,CA-2014-133424,US-2017-162558,US-2017-162558,US-2017-162558,US-2017-162558,US-2017-162558,US-2017-162558,US-2017-162558,CA-2017-169691,CA-2017-169691,CA-2017-169691,US-2017-132381,CA-2017-139822,CA-2017-139822,US-2014-112949,US-2014-112949
PurchaseDate,2017-06-08 00:00:00,2017-09-02 00:00:00,2014-07-01 00:00:00,2014-11-29 00:00:00,2016-06-29 00:00:00,2014-05-26 00:00:00,2016-01-15 00:00:00,2016-01-15 00:00:00,2016-01-15 00:00:00,2014-12-27 00:00:00,2014-12-27 00:00:00,2015-12-13 00:00:00,2015-12-13 00:00:00,2017-10-07 00:00:00,2017-10-07 00:00:00,2017-10-07 00:00:00,2017-10-07 00:00:00,2017-10-07 00:00:00,2014-10-11 00:00:00,2014-10-11 00:00:00,2017-09-07 00:00:00,2015-08-02 00:00:00,2015-08-02 00:00:00,2016-04-07 00:00:00,2015-11-09 00:00:00,2015-11-09 00:00:00,2014-03-30 00:00:00,2014-03-30 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-10-02 00:00:00,2017-06-15 00:00:00,2017-06-15 00:00:00,2017-06-15 00:00:00,2017-08-22 00:00:00,2017-09-15 00:00:00,2017-09-15 00:00:00,2014-06-20 00:00:00,2014-06-20 00:00:00
ShipDate,2017-06-12 00:00:00,2017-09-08 00:00:00,2014-07-06 00:00:00,2014-12-06 00:00:00,2016-07-03 00:00:00,2014-05-29 00:00:00,2016-01-19 00:00:00,2016-01-19 00:00:00,2016-01-19 00:00:00,2014-12-31 00:00:00,2014-12-31 00:00:00,2015-12-20 00:00:00,2015-12-20 00:00:00,2017-10-11 00:00:00,2017-10-11 00:00:00,2017-10-11 00:00:00,2017-10-11 00:00:00,2017-10-11 00:00:00,2014-10-16 00:00:00,2014-10-16 00:00:00,2017-09-09 00:00:00,2015-08-06 00:00:00,2015-08-06 00:00:00,2016-04-12 00:00:00,2015-11-14 00:00:00,2015-11-14 00:00:00,2014-04-04 00:00:00,2014-04-04 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-10-05 00:00:00,2017-06-18 00:00:00,2017-06-18 00:00:00,2017-06-18 00:00:00,2017-08-24 00:00:00,2017-09-21 00:00:00,2017-09-21 00:00:00,2014-06-27 00:00:00,2014-06-27 00:00:00
ShipMode,Standard Class,Standard Class,Second Class,Standard Class,Second Class,First Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,First Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,Standard Class,First Class,First Class,First Class,First Class,First Class,First Class,First Class,First Class,First Class,First Class,First Class,Standard Class,Standard Class,Standard Class,Standard Class
CustomerId,Dl-13600,Dl-13600,Dl-13600,Co-12640,Co-12640,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Co-12640,Co-12640,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Co-12640,Co-12640,Dl-13600,Dp-13240,Dp-13240,Dl-13600,Dl-13600,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Dp-13240,Co-12640,Co-12640
CustomerName,Dorris liebe,Dorris liebe,Dorris liebe,Corey-Lock,Corey-Lock,Dean percer,Dean percer,Dean percer,Dean percer,Corey-Lock,Corey-Lock,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Dean percer,Corey-Lock,Corey-Lock,Dorris liebe,Dean percer,Dean percer,Dorris liebe,Dorris liebe,Dean percer,Dean percer,Dean pe

In [21]:
# 3 CustomerName have issues about it: Dorris liebe, Corey-Lock and Dean percer
# Manually update customer names and IDs

df.loc[df['CustomerId'] == 'Dl-13600', 'CustomerId'] = 'DL-13600'
df.loc[df['CustomerId'] == 'Co-12640', 'CustomerId'] = 'CL-12640'
df.loc[df['CustomerId'] == 'Dp-13240', 'CustomerId'] = 'DP-13240'

df.loc[df['CustomerId'] == 'DL-13600', 'CustomerName'] = 'Dorris Liebe'
df.loc[df['CustomerId'] == 'CL-12640', 'CustomerName'] = 'Corey Lock'
df.loc[df['CustomerId'] == 'DP-13240', 'CustomerName'] = 'Dean Percer'


In [22]:
valid_id_pattern = r'^[A-Z]{2}-\d{5}$' # string starting with 2 uppercase, followed by a hyphen and exactly 5 digits

df[~df['CustomerId'].str.match(valid_id_pattern)].T

""
PurchaseId
PurchaseDate
ShipDate
ShipMode
CustomerId
CustomerName
Segment
Country
City
State


In [23]:
# trim whitespaces from 'CustomerName' values

df['CustomerName'] = df['CustomerName'].str.strip()

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PurchaseId    9994 non-null   object        
 1   PurchaseDate  9994 non-null   datetime64[ns]
 2   ShipDate      9994 non-null   datetime64[ns]
 3   ShipMode      9994 non-null   object        
 4   CustomerId    9994 non-null   object        
 5   CustomerName  9994 non-null   object        
 6   Segment       9994 non-null   object        
 7   Country       9994 non-null   object        
 8   City          9994 non-null   object        
 9   State         9994 non-null   object        
 10  PostalCode    9994 non-null   int64         
 11  Region        9994 non-null   object        
 12  ProductId     9994 non-null   object        
 13  Category      9994 non-null   object        
 14  SubCategory   9994 non-null   object        
 15  ProductName   9994 non-null   object  

In [25]:
df['Segment'].value_counts()

Segment
Consumer       5191
Corporate      3020
Home Office    1783
Name: count, dtype: int64

In [26]:
df['Country'].unique()

array(['United States'], dtype=object)

In [27]:
df['State'].nunique()

49

In [28]:
df['State'].unique()

array(['Kentucky', 'California', 'Florida', 'North Carolina',
       'Washington', 'Texas', 'Wisconsin', 'Utah', 'Nebraska',
       'Pennsylvania', 'Illinois', 'Minnesota', 'Michigan', 'Delaware',
       'Indiana', 'New York', 'Arizona', 'Virginia', 'Tennessee',
       'Alabama', 'South Carolina', 'Oregon', 'Colorado', 'Iowa', 'Ohio',
       'Missouri', 'Oklahoma', 'New Mexico', 'Louisiana', 'Connecticut',
       'New Jersey', 'Massachusetts', 'Georgia', 'Nevada', 'Rhode Island',
       'Mississippi', 'Arkansas', 'Montana', 'New Hampshire', 'Maryland',
       'District of Columbia', 'Kansas', 'Vermont', 'Maine',
       'South Dakota', 'Idaho', 'North Dakota', 'Wyoming',
       'West Virginia'], dtype=object)

In [29]:
df['Region'].unique()

array(['South', 'West', 'Central', 'East'], dtype=object)

In [30]:
df['Region'].value_counts()

Region
West       3203
East       2848
Central    2323
South      1620
Name: count, dtype: int64

In [31]:
df['Category'].unique()

array(['Furniture', 'Office Supplies', 'Technology'], dtype=object)

In [32]:
df['Category'].value_counts()

Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

In [33]:
df['SubCategory'].nunique()

17

In [34]:
df['SubCategory'].unique()

array(['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage',
       'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper',
       'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines',
       'Copiers'], dtype=object)

In [35]:
df['SubCategory'].value_counts()

SubCategory
Binders        1523
Paper          1370
Furnishings     957
Phones          889
Storage         846
Art             796
Accessories     775
Chairs          617
Appliances      466
Labels          364
Tables          319
Envelopes       254
Bookcases       228
Fasteners       217
Supplies        190
Machines        115
Copiers          68
Name: count, dtype: int64

In [36]:
df['ProductName'].nunique()

1850

In [37]:
df.columns

Index(['PurchaseId', 'PurchaseDate', 'ShipDate', 'ShipMode', 'CustomerId',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductId', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [38]:
df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
PurchaseId,9994,5009,CA-2017-100111,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PurchaseDate,9994,NaN,NaN,NaN,2016-04-30 00:07:12.259355648,2014-01-03 00:00:00,2015-05-23 00:00:00,2016-06-26 00:00:00,2017-05-14 00:00:00,2017-12-30 00:00:00,NaN
ShipDate,9994,NaN,NaN,NaN,2016-05-03 23:06:58.571142912,2014-01-07 00:00:00,2015-05-27 00:00:00,2016-06-29 00:00:00,2017-05-18 00:00:00,2018-01-05 00:00:00,NaN
ShipMode,9994,4,Standard Class,5968,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CustomerId,9994,793,WB-21850,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CustomerName,9994,793,William Brown,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Segment,9994,3,Consumer,5191,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,9994,1,United States,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,9994,531,New York City,915,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,9994,49,California,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# check if rows have duplicatedof PurchaseId and ProductId pairs ( for sql table ) 

df_filtered = df[df.duplicated(subset=['PurchaseId', 'ProductId'], keep=False)]

df_filtered.head(20).T

,350,352,430,431,1300,1301,3183,3184,3405,3406,6498,6500,7881,7882,9168,9169
PurchaseId,CA-2016-129714,CA-2016-129714,US-2016-123750,US-2016-123750,CA-2016-137043,CA-2016-137043,CA-2017-152912,CA-2017-152912,US-2014-150119,US-2014-150119,CA-2015-103135,CA-2015-103135,CA-2017-118017,CA-2017-118017,CA-2016-140571,CA-2016-140571
PurchaseDate,2016-09-01 00:00:00,2016-09-01 00:00:00,2016-04-15 00:00:00,2016-04-15 00:00:00,2016-12-23 00:00:00,2016-12-23 00:00:00,2017-11-09 00:00:00,2017-11-09 00:00:00,2014-04-23 00:00:00,2014-04-23 00:00:00,2015-07-24 00:00:00,2015-07-24 00:00:00,2017-12-03 00:00:00,2017-12-03 00:00:00,2016-03-15 00:00:00,2016-03-15 00:00:00
ShipDate,2016-09-03 00:00:00,2016-09-03 00:00:00,2016-04-21 00:00:00,2016-04-21 00:00:00,2016-12-25 00:00:00,2016-12-25 00:00:00,2017-11-12 00:00:00,2017-11-12 00:00:00,2014-04-27 00:00:00,2014-04-27 00:00:00,2015-07-28 00:00:00,2015-07-28 00:00:00,2017-12-06 00:00:00,2017-12-06 00:00:00,2016-03-19 00:00:00,2016-03-19 00:00:00
ShipMode,First Class,First Class,Standard Class,Standard Class,Second Class,Second Class,Second Class,Second Class,Standard Class,Standard Class,Standard Class,Standard Class,Second Class,Second Class,Standard Class,Standard Class
CustomerId,AB-10060,AB-10060,RB-19795,RB-19795,LC-17140,LC-17140,BM-11650,BM-11650,LB-16795,LB-16795,SS-20515,SS-20515,LC-16870,LC-16870,SJ-20125,SJ-20125
CustomerName,Adam Bellavance,Adam Bellavance,Ross Baird,Ross Baird,Logan Currie,Logan Currie,Brian Moss,Brian Moss,Laurel Beltran,Laurel Beltran,Shirley Schmidt,Shirley Schmidt,Lena Cacioppo,Lena Cacioppo,Sanjit Jacobs,Sanjit Jacobs
Segment,Home Office,Home Office,Home Office,Home Office,Consumer,Consumer,Corporate,Corporate,Home Office,Home Office,Home Office,Home Office,Consumer,Consumer,Home Office,Home Office
Country,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
City,New York City,New York City,Gastonia,Gastonia,Springfield,Springfield,Columbia,Columbia,Columbus,Columbus,Louisville,Louisville,Thornton,Thornton,Jackson,Jackson
State,New York,New York,North Carolina,North Carolina,Virginia,Virginia,Maryland,Maryland,Ohio,Ohio,Kentucky,Kentucky,Colorado,Colorado,Mississippi,Mississippi


In [40]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 350 to 9169
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PurchaseId    16 non-null     object        
 1   PurchaseDate  16 non-null     datetime64[ns]
 2   ShipDate      16 non-null     datetime64[ns]
 3   ShipMode      16 non-null     object        
 4   CustomerId    16 non-null     object        
 5   CustomerName  16 non-null     object        
 6   Segment       16 non-null     object        
 7   Country       16 non-null     object        
 8   City          16 non-null     object        
 9   State         16 non-null     object        
 10  PostalCode    16 non-null     int64         
 11  Region        16 non-null     object        
 12  ProductId     16 non-null     object        
 13  Category      16 non-null     object        
 14  SubCategory   16 non-null     object        
 15  ProductName   16 non-null     object       

* records appear to be legitimate, with additional items added to the same purchase on the same day. We should therefore sum the values and consolidate them into 6 unique pairs of PurchaseId and ProductId.

In [41]:
# list of index pairs to process
index_pairs = [
    (350, 352),
    (1300, 1301),
    (3183, 3184),
    (3405, 3406),
    (6498, 6500),
    (7881, 7882),
    (430, 431),
    (9168, 9169)
]

# order indices in descending order 
index_pairs = sorted(index_pairs, reverse=True)

# process each pair of indices
for idx1, idx2 in index_pairs:
    if idx1 in df.index and idx2 in df.index:
        # Extract values
        row_1 = df.loc[idx1]
        row_2 = df.loc[idx2]
        
        # Update the second row with the values from the first row (sum)
        df.at[idx2, 'Quantity'] += row_1['Quantity']
        df.at[idx2, 'Sales'] += row_1['Sales']
        df.at[idx2, 'Profit'] += row_1['Profit']  # add the sum of Profit
        
        # drop the first idx1 row
        df = df.drop(idx1)



In [42]:
df_filtered = df[df.duplicated(subset=['PurchaseId', 'ProductId'], keep=False)]

df_filtered.head()

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit


In [43]:
df[df['PurchaseId'] == 'US-2014-150119']

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
3406,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,562.744,4,0.3,-24.1176
3407,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,OFF-BI-10000145,Office Supplies,Binders,Zipper Ring Binder Pockets,7.488,8,0.7,-5.2416
3408,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-FU-10002191,Furniture,Furnishings,G.E. Halogen Desk Lamp Bulbs,22.336,4,0.2,7.8176


### 3: Create new columns

In [44]:
# CatalogPrice
df['CatalogPrice'] = (df['Sales'] / (df['Quantity'] * (1 - df['Discount']))).round(2)

# SupplierPrice
df['SupplierPrice'] = (df['CatalogPrice'] - (df['Profit'] / df['Quantity'])).round(2)

# ensure no negative SupplierPrice values
df.loc[df['SupplierPrice'] < 0, 'SupplierPrice'] = 0

In [45]:
# check the sales with loss

df_filtered = df[df['SupplierPrice'] > df['CatalogPrice']]
df_filtered.shape

(1870, 22)

In [46]:
df_filtered.describe()

,PurchaseDate,ShipDate,PostalCode,Sales,Quantity,Discount,Profit,CatalogPrice,SupplierPrice
count,1870,1870,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000
mean,2016-04-27 12:28:29.518716672,2016-05-01 11:48:26.951871744,55997.947594,250.645537,3.764706,0.480984,-83.492666,116.828053,138.132406
min,2014-01-04 00:00:00,2014-01-08 00:00:00,1841.000000,0.444000,1.000000,0.100000,-6599.978000,1.260000,1.460000
25%,2015-05-10 12:00:00,2015-05-15 12:00:00,33024.000000,12.482500,2.000000,0.200000,-58.674075,11.485000,14.380000
50%,2016-06-13 00:00:00,2016-06-17 00:00:00,60623.000000,71.029000,3.000000,0.400000,-18.112200,44.430000,54.295000
75%,2017-05-05 18:00:00,2017-05-10 00:00:00,77095.000000,285.552000,5.000000,0.700000,-6.249250,135.230000,157.285000
max,2017-12-30 00:00:00,2018-01-03 00:00:00,98198.000000,22638.480000,14.000000,0.800000,-0.089500,7546.160000,7848.010000
std,NaN,NaN,26046.794089,715.294305,2.141150,0.235106,284.493224,279.477709,330.527801


* all records which result losses are discunted prices

In [47]:
df.duplicated().sum()

0

In [48]:
df = df.sort_values(by=['PurchaseDate', 'CustomerId']).reset_index(drop=True)
df.head()

,PurchaseId,PurchaseDate,ShipDate,ShipMode,CustomerId,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductId,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit,CatalogPrice,SupplierPrice
0,CA-2014-103800,2014-01-03,2014-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512,10.28,7.50
1,CA-2014-112326,2014-01-04,2014-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717,4.91,3.49
2,CA-2014-112326,2014-01-04,2014-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748,113.64,135.23
3,CA-2014-112326,2014-01-04,2014-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870,8.85,11.59
4,CA-2014-141817,2014-01-05,2014-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840,8.14,6.51


### 4: Create data frames for SQL tables

In [49]:
# PurchaseDetail DataFrame

df_PurchaseDetail = df[['PurchaseId', 'ProductId', 'Quantity', 'Sales', 'Discount', 'Profit', 'ShipDate', 'ShipMode']]
df_PurchaseDetail = df_PurchaseDetail.sort_values(by=['PurchaseId', 'ProductId'], ascending=[False, True])

# df_Purchase DataFrame - Group by PurchaseId
df_Purchase = df.groupby('PurchaseId').first().reset_index()[['PurchaseId', 'CustomerId', 'PurchaseDate']]

# Sort by PurchaseDate (latest first) and CustomerId
df_Purchase = df_Purchase.sort_values(by=['PurchaseDate', 'CustomerId'], ascending=[False, True])

# df_Customer DataFrame - Group by CustomerId

df_Customer = df.groupby('CustomerId').first().reset_index()[['CustomerId', 'CustomerName', 'Segment']]
df_Customer = df_Customer.sort_values(by=['CustomerId', 'CustomerName'])

# Drop duplicates to get each unique pair of CustomerId and PostalCode

df_CustomerLocation = df.drop_duplicates(subset=['CustomerId', 'PostalCode'], keep='first')[['CustomerId', 'PostalCode', 'City', 'Region', 'State', 'Country']]
df_CustomerLocation = df_CustomerLocation.sort_values(by=['CustomerId', 'PostalCode'])

# Drop duplicates to get the unique pair of ProductId and SubCategory

df_Category = df.drop_duplicates(subset=['ProductId', 'SubCategory'], keep='first')[['ProductId', 'Category', 'SubCategory']]
df_Category = df_Category.sort_values(by=['ProductId', 'SubCategory'])

# df_Product DataFrame - Use most recent values for CatalogPrice and SupplierPrice
# Group by ProductId and PurchaseDate to get the latest rows based on PurchaseDate

df_Product = df.groupby('ProductId').apply(lambda x: x.sort_values('PurchaseDate').iloc[-1]).reset_index(drop=True)[['ProductId', 'ProductName', 'CatalogPrice', 'SupplierPrice']]
df_Product = df_Product.sort_values(by=['ProductId', 'ProductName'])

In [50]:
df_PurchaseDetail.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9986 entries, 7904 to 989
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   PurchaseId  9986 non-null   object        
 1   ProductId   9986 non-null   object        
 2   Quantity    9986 non-null   int64         
 3   Sales       9986 non-null   float64       
 4   Discount    9986 non-null   float64       
 5   Profit      9986 non-null   float64       
 6   ShipDate    9986 non-null   datetime64[ns]
 7   ShipMode    9986 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 702.1+ KB


In [51]:
df_Purchase.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5009 entries, 3328 to 56
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PurchaseId    5009 non-null   object        
 1   CustomerId    5009 non-null   object        
 2   PurchaseDate  5009 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 156.5+ KB


In [52]:
df_Customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerId    793 non-null    object
 1   CustomerName  793 non-null    object
 2   Segment       793 non-null    object
dtypes: object(3)
memory usage: 18.7+ KB


In [53]:
df_CustomerLocation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4910 entries, 1105 to 958
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerId  4910 non-null   object
 1   PostalCode  4910 non-null   int64 
 2   City        4910 non-null   object
 3   Region      4910 non-null   object
 4   State       4910 non-null   object
 5   Country     4910 non-null   object
dtypes: int64(1), object(5)
memory usage: 268.5+ KB


In [54]:
df_Product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ProductId      1862 non-null   object 
 1   ProductName    1862 non-null   object 
 2   CatalogPrice   1862 non-null   float64
 3   SupplierPrice  1862 non-null   float64
dtypes: float64(2), object(2)
memory usage: 58.3+ KB


In [55]:
df_Category.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1862 entries, 8373 to 10
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductId    1862 non-null   object
 1   Category     1862 non-null   object
 2   SubCategory  1862 non-null   object
dtypes: object(3)
memory usage: 58.2+ KB


### 5: Create and populate tables

In [56]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# SQL commands to create tables 
create_tables_sql = ''' 
-- Create the Customer table 
CREATE TABLE IF NOT EXISTS Customer (
    CustomerId TEXT PRIMARY KEY,                
    CustomerName TEXT NOT NULL, 
    Segment TEXT NOT NULL
);

-- Create the CustomerLocation table
CREATE TABLE IF NOT EXISTS CustomerLocation (
    CustomerId TEXT NOT NULL, 
    PostalCode TEXT NOT NULL,
    City TEXT NOT NULL,
    Region TEXT,
    State TEXT NOT NULL,
    Country TEXT NOT NULL,
    PRIMARY KEY (PostalCode, CustomerId),
    FOREIGN KEY (CustomerId) REFERENCES Customer(CustomerId)
);

-- Create the Purchase table
CREATE TABLE IF NOT EXISTS Purchase (
    PurchaseId TEXT PRIMARY KEY,                
    CustomerId TEXT NOT NULL, 
    PurchaseDate DATE NOT NULL,
    FOREIGN KEY (CustomerId) REFERENCES Customer(CustomerId)
);

-- Create the Product table
CREATE TABLE IF NOT EXISTS Product (
    ProductId TEXT PRIMARY KEY,
    ProductName TEXT NOT NULL,
    CatalogPrice DECIMAL NOT NULL,
    SupplierPrice DECIMAL NOT NULL
);

-- Create the PurchaseDetail table
CREATE TABLE IF NOT EXISTS PurchaseDetail (
    PurchaseId TEXT NOT NULL,
    ProductId TEXT NOT NULL,
    Quantity INTEGER NOT NULL,
    Sales DECIMAL NOT NULL,
    Discount DECIMAL NOT NULL,
    Profit DECIMAL NOT NULL,
    ShipDate DATE,
    ShipMode TEXT,
    PRIMARY KEY (PurchaseId, ProductId),
    FOREIGN KEY (PurchaseId) REFERENCES Purchase(PurchaseId),
    FOREIGN KEY (ProductId) REFERENCES Product(ProductId)
);

-- Create the Category table
CREATE TABLE IF NOT EXISTS Category (
    ProductId TEXT NOT NULL,
    SubCategory TEXT NOT NULL,
    Category TEXT NOT NULL,
    PRIMARY KEY (ProductId, SubCategory),
    FOREIGN KEY (ProductId) REFERENCES Product(ProductId)
);
'''

# Execute the SQL commands to create tables
cursor.executescript(create_tables_sql)

# Commit changes and close the connection
conn.commit()

print("Database and tables created successfully.")

Database and tables created successfully.


In [57]:
# Populate data to tables
df_Customer.to_sql('Customer', conn, if_exists='append', index=False)
df_CustomerLocation.to_sql('CustomerLocation', conn, if_exists='append', index=False)
df_Purchase.to_sql('Purchase', conn, if_exists='append', index=False)
df_PurchaseDetail.to_sql('PurchaseDetail', conn, if_exists='append', index=False)
df_Product.to_sql('Product', conn, if_exists='append', index=False)
df_Category.to_sql('Category', conn, if_exists='append', index=False)


1862

* run some queries

In [58]:
# view categories by total revenue

query = """
SELECT Category.Category, 
       SUM(PurchaseDetail.Sales) AS TotalRevenue
FROM PurchaseDetail
JOIN Product ON PurchaseDetail.ProductId = Product.ProductId
JOIN Category ON Product.ProductId = Category.ProductId
GROUP BY Category.Category
ORDER BY TotalRevenue DESC;
"""

# execute the query and fetch the results into a DataFrame
pd.read_sql_query(query, conn)

,Category,TotalRevenue
0,Technology,836154.0330
1,Furniture,741999.7953
2,Office Supplies,719047.0320


In [59]:
# view top 10 ProductName by reveneu

query = """
SELECT Product.ProductName, 
       SUM(PurchaseDetail.Profit) AS TotalProfit
FROM PurchaseDetail
JOIN Product ON PurchaseDetail.ProductId = Product.ProductId
JOIN Category ON Product.ProductId = Category.ProductId
WHERE Category.Category = 'Technology'
GROUP BY Product.ProductName
ORDER BY TotalProfit DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,ProductName,TotalProfit
0,Canon imageCLASS 2200 Advanced Copier,25199.9280
1,Hewlett Packard LaserJet 3310 Copier,6983.8836
2,Canon PC1060 Personal Laser Copier,4570.9347
3,Logitech G19 Programmable Gaming Keyboard,4425.3432
4,HP Designjet T520 Inkjet Large Format Printer ...,4094.9766
5,Ativa V4110MDD Micro-Cut Shredder,3772.9461
6,"3D Systems Cube Printer, 2nd Generation, Magenta",3717.9714
7,Zebra ZM400 Thermal Label Printer,3343.5360
8,Hewlett Packard 610 Color Digital Copier / Pri...,3124.9375
9,Plantronics CS510 - Over-the-Head monaural Wir...,3085.0325


In [60]:
conn.close()
